# In-Memory OLTP
Následující sada příkladů slouží k ukázce nasazení In-Memory objektů v MS SQL databázi. Provedeny jsou následující kroky: <br/>
1. Vytvoření prázdné databáze<br/>
2. Vytvoření memory optimized filegroup (pro odkládání in-memory dat)<br/>
3. Vytvoření in-memory tabulky<br/>
4. Vytvoření nativně kompilované procedury<br/>
    4.1. Tabulkový datový typ jako parameter (také in-memory)<br/>
    4.2. Procedura používající tento tabulkový typ<br/>
    4.3. Vygenerování 1 mio testovacích záznamů<br/>
5. Vytvoření procedury pro odkládání dat z in-memory objektů na disk (snapshot isolation level) 

## Vyčištění prostředí
Chceme začít načisto

In [12]:
USE master
GO

DROP DATABASE IF EXISTS InMemDemo
GO

Commands completed successfully.

Total execution time: 00:00:00.0281192

Commands completed successfully.

Total execution time: 00:00:02.1216756

## In-Memory OLTP v praxi; vytvoření prázdné databáze
Databáze jako každá jiná, žádné extra konfigurace

In [2]:
CREATE DATABASE InMemDemo
GO
ALTER DATABASE InMemDemo SET RECOVERY SIMPLE;
GO

Commands completed successfully.

Total execution time: 00:00:01.2631369

Commands completed successfully.

Total execution time: 00:00:00.0660121

## In-Memory OLTP v praxi; přidání memory optimized FG

Memory optimized filegroup je ve skutečnosti filestream, do kterého SQL Server při checkpointu (mimo uživatelskou transakci) ukládá *dump* paměťových objektů

In [3]:
ALTER DATABASE InMemDemo ADD FILEGROUP INMEM
CONTAINS MEMORY_OPTIMIZED_DATA
GO

Commands completed successfully.

Total execution time: 00:00:00.0378410

### Folder do filegroup
...jinak by FG nedávala smysl

In [4]:
ALTER DATABASE InMemDemo ADD FILE
(
    NAME = 'InMemFile'
    , FILENAME = 'D:\InMemFs'

) TO FILEGROUP  INMEM

Commands completed successfully.

Total execution time: 00:00:00.1354943

TEĎ JE DATABÁZE NACHYSTANÁ NA PAMĚŤOVÉ OBJEKTY

## Vytvoření in-memory tabulky s dodržením ACID vlastností
Tabulka je tzv. *full durable*, protože má nastavenu vlastnost DURABILITY = SCHEMA_AND_DATA
<br/>
Index je "HASH", počet kapes v indexu je násobek čísla 2^20.  

In [5]:
USE InMemDemo
GO
DROP TABLE IF EXISTS dbo.InMemCisla
GO
CREATE TABLE dbo.InMemCisla
(
    Id int not null primary key nonclustered hash with (bucket_count=2097152)
    , DatumACas datetime2 not null default(sysdatetime())
    , NahodneCislo dec(10,4) not null
) with (memory_optimized = on, durability = schema_and_data)

Commands completed successfully.

Total execution time: 00:00:00.0249657

Commands completed successfully.

Total execution time: 00:00:00.0249396

Commands completed successfully.

Total execution time: 00:00:04.2047485

## Vytvoření memory optimized datového typu
Bude sloužit jako parametr uložené procedury

In [6]:
USE InMemDemo
GO

CREATE TYPE dbo.InMemType
AS TABLE
(
    Id int not null primary key nonclustered hash with (bucket_count = 1048576)
    , NahodneCislo dec(10,4)
)
WITH (MEMORY_OPTIMIZED = ON)

Commands completed successfully.

Total execution time: 00:00:00.0238900

Commands completed successfully.

Total execution time: 00:00:00.5359755

## Nativně kompilovaná procedura (zatím) bez parametru
Procedura pro ilustraci rychlosti generování 1.000.000 náhodných záznamů.

In [7]:
USE InMemDemo
GO

CREATE OR ALTER PROC dbo.procInMemRandomNumbers
WITH NATIVE_COMPILATION, SCHEMABINDING
AS
BEGIN ATOMIC WITH (TRANSACTION ISOLATION LEVEL = SNAPSHOT, LANGUAGE = 'us_english')
    DECLARE @i int = 1
    WHILE @i <= 1000000
     BEGIN
        INSERT dbo.InMemCisla (Id, NahodneCislo) VALUES (@i, RAND() * 1000)
        SET @i += 1
     END
END

Commands completed successfully.

Total execution time: 00:00:00.0264396

Commands completed successfully.

Total execution time: 00:00:00.5241079

... a teď spustit a měřit...

In [9]:
USE InMemDemo
GO
DELETE dbo.InMemCisla
GO
EXEC dbo.procInMemRandomNumbers
GO

SELECT COUNT(*) FROM dbo.InMemCisla
GO

Commands completed successfully.

Total execution time: 00:00:00.0242152

(1000000 rows affected)

Total execution time: 00:00:01.2273445

Commands completed successfully.

Total execution time: 00:00:05.2078878

(1 row affected)

Total execution time: 00:00:00.0585396

(No column name)
1000000


## Příklad s tabulkovým parametrem
V reálném nasazení byl použit tabulkový parametr, který obsahoval záznamy načtené ze zdroje. 

In [10]:
USE InMemDemo
GO

CREATE OR ALTER PROC dbo.procInMemRandomNumbersPar
    @data dbo.InMemType readonly
WITH NATIVE_COMPILATION, SCHEMABINDING
AS
BEGIN ATOMIC WITH (TRANSACTION ISOLATION LEVEL = SNAPSHOT, LANGUAGE = 'us_english')
    DECLARE @start time = sysdatetime()
    INSERT dbo.InMemCisla (Id, NahodneCislo) SELECT Id, NahodneCislo FROM @data
    DECLARE @stop time = sysdatetime()
    SELECT datediff(microsecond, @start, @stop) as CommandMicroseconds
END

Commands completed successfully.

Total execution time: 00:00:00.0242618

Commands completed successfully.

Total execution time: 00:00:00.5384137

## Test spuštění
Čísla jsou vygenerována napřed do proměnné (obdoba naštení dat z výrobní linky), pak procedura záznamy jen zpracuje.


In [11]:
USE InMemDemo
GO
SET NOCOUNT ON
DELETE dbo.InMemCisla
GO
DECLARE @data dbo.InMemType
DECLARE @i int = 1
WHILE @i <= 1000000
 BEGIN
    INSERT @data (Id, NahodneCislo) VALUES (@i, RAND() * 1000)
    SET @i += 1
 END
EXEC dbo.procInMemRandomNumbersPar @data

Commands completed successfully.

Total execution time: 00:00:00.0247312

Commands completed successfully.

Total execution time: 00:00:01.2162307

Commands completed successfully.

Total execution time: 00:00:18.1396843

CommandMicroseconds
1093776


## Odklání dat do diskové tabulky
Nejpodstatnější je v explicitní transakci hint *snapshot*! Následující kód se nebude spouštět (tabulka dbo.tabulkaOnDisk neexistuje). Následující kód neobsahuje kompletní ošetření chybových stavů.

In [ ]:
USE InMemDemo
GO
BEGIN TRAN
    INSERT dbo.tabulkaOnDisk SELECT * FROM dbo.InMemCisla (snapshot)
    DELETE dbo.InMemCisla (snapshot)
COMMIT